# ![s](https://s19.postimg.org/97wj4l35f/screenshot_1489112218.png)

# This tutorial is for SQLAlchemy 1.1.6

In [1]:
import sqlite3
from sqlalchemy import *
#The Python SQL Toolkit and Object Relational Mapper(ORM)
#http://docs.sqlalchemy.org/en/latest/intro.html#installation
#http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [2]:
import sqlalchemy
print (sqlalchemy.__version__)

1.1.6


In [3]:
db_engine = create_engine('sqlite:///sqlalchemy_example.db')
#or
#db_engine = create_engine('sqlite:///sqlalchemy_example.db',echo=True)

In [10]:
db_engine.echo=True 
#I want to see whats going in the background

In [6]:
metadata =MetaData(db_engine)

In [23]:
'''import pip
for i in pip.get_installed_distributions(local_only=True):
    print(i)'''

'import pip\nfor i in pip.get_installed_distributions(local_only=True):\n    print(i)'

In [7]:
user=Table('users',metadata,
          Column('ID',Integer,primary_key=True),
          Column('name', String(40)),
          Column('age', Integer)
          )#Here table name is users

In [ ]:
user.create()

### Intersting into User

In [28]:
user = Table('users', metadata, autoload=True)

In [26]:
i=user.insert()

In [29]:
i.execute(name='ABC', age=20)

2017-03-09 20:58:48,311 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2017-03-09 20:58:48,315 INFO sqlalchemy.engine.base.Engine ('ABC', 20)


INFO:sqlalchemy.engine.base.Engine:('ABC', 20)


2017-03-09 20:58:48,319 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [30]:
i.execute({'name': 'Superman', 'age': 30},
          {'name': 'Batman', 'age': 57},
          {'name': 'Acquaman', 'age': 38},
          {'name': 'Wonderwoman', 'age': 200},
          {'name': 'Flash', 'age': 25},
          {'name': 'Cyborg', 'age': 35}
         )

2017-03-09 21:00:45,361 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2017-03-09 21:00:45,362 INFO sqlalchemy.engine.base.Engine (('Superman', 30), ('Batman', 57), ('Acquaman', 38), ('Wonderwoman', 200), ('Flash', 25), ('Cyborg', 35))


INFO:sqlalchemy.engine.base.Engine:(('Superman', 30), ('Batman', 57), ('Acquaman', 38), ('Wonderwoman', 200), ('Flash', 25), ('Cyborg', 35))


2017-03-09 21:00:45,365 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [44]:
s=user.select()

In [45]:
conn=s.execute()

### Fetch one row, just like DB-API ``cursor.fetchone()``.

In [46]:
allrows=conn.fetchone()

In [47]:
print("ID: ", allrows[0])
print("Name: ", allrows.name)
print("Age: ", allrows['age'])

ID:  1
Name:  ABC
Age:  20


### Show all records in DB

In [48]:
for allrows in conn:
    print(allrows.name, 'is',allrows.age, 'years old')

Superman is 30 years old
Batman is 57 years old
Acquaman is 38 years old
Wonderwoman is 200 years old
Flash is 25 years old
Cyborg is 35 years old


## Echo Off

In [49]:
db_engine.echo=False

In [50]:
s=user.select()
conn=s.execute()
allrows=conn.fetchone()
for allrows in conn:
    print(allrows.name, 'is',allrows.age, 'years old')

Superman is 30 years old
Batman is 57 years old
Acquaman is 38 years old
Wonderwoman is 200 years old
Flash is 25 years old
Cyborg is 35 years old


### Defining Run

In [51]:
def run(stmt):
    conn = stmt.execute()
    for rows in conn:
        print (rows)

## Finding names with 'man'

In [52]:
s=user.select(user.c.name.like('%man%'))# Most WHERE clauses can be constructed via normal comparisons
run(s)

(2, 'Superman', 30)
(3, 'Batman', 57)
(4, 'Acquaman', 38)
(5, 'Wonderwoman', 200)


## Finding age more than 50

In [71]:
s=user.select(user.c.age>50)
run(s)

(3, 'Batman', 57)
(5, 'Wonderwoman', 200)


## Python keywords like "and", "or", and "not" can't be overloaded, so
SQLAlchemy uses functions instead

In [79]:
s=user.select(and_(user.c.age>50,user.c.name.like('___ma%')))
run(s)

(3, 'Batman', 57)


### We can use &, | and ~ -- but watch out for priority!!!


In [82]:
s=user.select((user.c.age>50) & (user.c.name.like('___ma%')))
run(s)

(3, 'Batman', 57)


In [11]:
s = user.select(user.c.name.endswith('h'))
run(s)

(6, 'Flash', 25)


### Our original data in SQLite

In [53]:
s = user.select(user.c.name.like('%%'))
run(s)

(1, 'ABC', 20)
(2, 'Superman', 30)
(3, 'Batman', 57)
(4, 'Acquaman', 38)
(5, 'Wonderwoman', 200)
(6, 'Flash', 25)
(7, 'Cyborg', 35)


# Plumbing data from PANDAS to SQLite

## ![s](http://pbpython.com/images/pandas-dataframe-shadow.png)

In [12]:
import pandas as pd

In [13]:
col=['ID','Name','Age']

In [18]:
data=[(8,'Ironman',58),
     (9,'Thor',300),
     (10,'Hulk',55)]

In [19]:
df=pd.DataFrame(data,columns=col)

In [61]:
df.head()#This dataframe we will store in our SQLite

,ID,Name,Age
0,8,Ironman,58
1,9,Thor,300
2,10,Hulk,55


# From PANDAS to SQLite

In [56]:
df.to_sql('users',db_engine, if_exists='append',index=False)

In [57]:
s = user.select(user.c.name.like('%%'))
run(s)

(1, 'ABC', 20)
(2, 'Superman', 30)
(3, 'Batman', 57)
(4, 'Acquaman', 38)
(5, 'Wonderwoman', 200)
(6, 'Flash', 25)
(7, 'Cyborg', 35)
(8, 'Ironman', 58)
(9, 'Thor', 300)
(10, 'Hulk', 55)


# From SQLite to PANDAS

In [58]:
df2=pd.read_sql_query('SELECT * FROM users',db_engine)#No need to use flavor...bc we are using SQLite

In [59]:
df2

,ID,name,age
0,1,ABC,20
1,2,Superman,30
2,3,Batman,57
3,4,Acquaman,38
4,5,Wonderwoman,200
5,6,Flash,25
6,7,Cyborg,35
7,8,Ironman,58
8,9,Thor,300
9,10,Hulk,55


### Reference : https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/
http://www.rmunn.com/sqlalchemy-tutorial/tutorial.html